# Setup

In [ ]:
%pip install unidecode

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
  Obtaining dependency information for unidecode from https://files.pythonhosted.org/packages/84/b7/6ec57841fb67c98f52fc8e4a2d96df60059637cba077edc569a302a8ffc7/Unidecode-1.3.8-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/235.5 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━ 174.1/235.5 kB 5.0 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.9 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [ ]:
from pyspark.sql.functions import when, avg
from pyspark.sql.types import IntegerType, DecimalType
from pyspark.sql.functions import col, regexp_replace
import re
from unidecode import unidecode

spark._jsc.hadoopConfiguration().set("fs.s3a.access.key", "<access_key>")
spark._jsc.hadoopConfiguration().set(
    "fs.s3a.secret.key", "<secret_key>"
)
spark._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "s3.amazonaws.com")

# ETL Enade File

In [ ]:
s3_path = "s3://ivan-mvp-puc/conceito_enade_2022.CSV"
df = (
    spark.read.format("csv")
    .option("header", "true")
    .option("delimiter", ";")
    .option("encoding", "ISO-8859-1")
    .load(s3_path)
)
print(df.columns)

['Ano', 'Código da Área', 'Área de Avaliação', 'Grau Acadêmico', 'Código da IES', 'Nome da IES*', 'Sigla da IES*', 'Organização Acadêmica', 'Categoria Administrativa', 'Código do Curso', 'Modalidade de Ensino', 'Código do Município**', 'Município do Curso**', 'Sigla da UF** ', 'Nº de Concluintes Inscritos', 'Nº  de Concluintes Participantes', 'Nota Bruta - FG', 'Nota Padronizada - FG', 'Nota Bruta - CE', 'Nota Padronizada - CE', 'Conceito Enade (Contínuo)', 'Conceito Enade (Faixa)', 'Observação']


In [ ]:
df = df.drop("Observação")

It is necessary to remove any special characters and spaces from the column names, that's what the next function does

In [ ]:
def clean_column_name(col_name):
    cleaned_name = unidecode(col_name).lower()
    cleaned_name = re.sub(r"[^a-zA-Z0-9_]", "_", cleaned_name)
    cleaned_name = re.sub(r"_+", "_", cleaned_name)
    cleaned_name = re.sub(r"_+$", "", cleaned_name)

    return cleaned_name

In [ ]:
new_column_names = [clean_column_name(col_name) for col_name in df.columns]
for old_name, new_name in zip(df.columns, new_column_names):
    df = df.withColumnRenamed(old_name, new_name)

When the course did not have enouth data to have an avaluation, the value of the enade column was "SC", to make it standard and make the column to be an integer, the value "SC" (witch translates to "Without Avaluation") was substituted to a null value

In [ ]:
df = df.withColumn(
    "conceito_enade_faixa",
    when((df["conceito_enade_faixa"] == "SC"), None).otherwise(
        df["conceito_enade_faixa"]
    ),
)

Then, the columns were transformed to their correct data types

In [ ]:
new_types = {
    "ano": IntegerType(),
    "codigo_da_area": IntegerType(),
    "codigo_da_ies": IntegerType(),
    "codigo_do_curso": IntegerType(),
    "codigo_do_municipio": IntegerType(),
    "no_de_concluintes_inscritos": IntegerType(),
    "no_de_concluintes_participantes": IntegerType(),
    "nota_bruta_fg": DecimalType(10, 3),
    "nota_padronizada_fg": DecimalType(10, 3),
    "nota_bruta_ce": DecimalType(10, 3),
    "nota_padronizada_ce": DecimalType(10, 3),
    "conceito_enade_continuo": DecimalType(10, 3),
    "conceito_enade_faixa": IntegerType(),
}
for col_name, col_type in new_types.items():
    if isinstance(col_type, DecimalType):
        df = df.withColumn(col_name, regexp_replace(col(col_name), ",", "."))
    df = df.withColumn(col_name, col(col_name).cast(col_type))

df.printSchema()

root
 |-- ano: integer (nullable = true)
 |-- codigo_da_area: integer (nullable = true)
 |-- area_de_avaliacao: string (nullable = true)
 |-- grau_academico: string (nullable = true)
 |-- codigo_da_ies: integer (nullable = true)
 |-- nome_da_ies: string (nullable = true)
 |-- sigla_da_ies: string (nullable = true)
 |-- organizacao_academica: string (nullable = true)
 |-- categoria_administrativa: string (nullable = true)
 |-- codigo_do_curso: integer (nullable = true)
 |-- modalidade_de_ensino: string (nullable = true)
 |-- codigo_do_municipio: integer (nullable = true)
 |-- municipio_do_curso: string (nullable = true)
 |-- sigla_da_uf: string (nullable = true)
 |-- no_de_concluintes_inscritos: integer (nullable = true)
 |-- no_de_concluintes_participantes: integer (nullable = true)
 |-- nota_bruta_fg: decimal(10,3) (nullable = true)
 |-- nota_padronizada_fg: decimal(10,3) (nullable = true)
 |-- nota_bruta_ce: decimal(10,3) (nullable = true)
 |-- nota_padronizada_ce: decimal(10,3) (nul

The dataset was then, normalized. Separating between the institution data, the city data, the course area data and the course data

In [ ]:
ies_df = df.select(
    "codigo_da_ies",
    "nome_da_ies",
    "sigla_da_ies",
    "organizacao_academica",
    "categoria_administrativa",
).distinct()

municipio_df = df.select("codigo_do_municipio", "municipio_do_curso").distinct()

area_df = df.select("codigo_da_area", "area_de_avaliacao").distinct()

curso_df = df.select(
    "ano",
    "codigo_do_curso",
    "codigo_da_area",
    "grau_academico",
    "codigo_da_ies",
    "codigo_do_municipio",
    "modalidade_de_ensino",
    "no_de_concluintes_inscritos",
    "no_de_concluintes_participantes",
    "nota_bruta_fg",
    "nota_padronizada_fg",
    "nota_bruta_ce",
    "nota_padronizada_ce",
    "conceito_enade_continuo",
    "conceito_enade_faixa",
).distinct()

# ETL - Student Questionnaire

In [ ]:
s3_path = "s3://ivan-mvp-puc/microdados2022_arq4.txt"
sq_df = (
    spark.read.format("csv")
    .option("header", "true")
    .option("delimiter", ";")
    .load(s3_path)
)
sq_df.printSchema()

root
 |-- NU_ANO: string (nullable = true)
 |-- CO_CURSO: string (nullable = true)
 |-- QE_I27: string (nullable = true)
 |-- QE_I28: string (nullable = true)
 |-- QE_I29: string (nullable = true)
 |-- QE_I30: string (nullable = true)
 |-- QE_I31: string (nullable = true)
 |-- QE_I32: string (nullable = true)
 |-- QE_I33: string (nullable = true)
 |-- QE_I34: string (nullable = true)
 |-- QE_I35: string (nullable = true)
 |-- QE_I36: string (nullable = true)
 |-- QE_I37: string (nullable = true)
 |-- QE_I38: string (nullable = true)
 |-- QE_I39: string (nullable = true)
 |-- QE_I40: string (nullable = true)
 |-- QE_I41: string (nullable = true)
 |-- QE_I42: string (nullable = true)
 |-- QE_I43: string (nullable = true)
 |-- QE_I44: string (nullable = true)
 |-- QE_I45: string (nullable = true)
 |-- QE_I46: string (nullable = true)
 |-- QE_I47: string (nullable = true)
 |-- QE_I48: string (nullable = true)
 |-- QE_I49: string (nullable = true)
 |-- QE_I50: string (nullable = true)
 |-- 

The same treatment in the column name was applied here

In [ ]:
new_column_names = [clean_column_name(col_name) for col_name in sq_df.columns]
for old_name, new_name in zip(df.columns, new_column_names):
    sq_df = sq_df.withColumnRenamed(old_name, new_name)

This file has each unique response to the student questionnaire, the aswers varied from 1 (Strongly Disagree) to 6 (Strongly Agree). The numbers 7 and 8 are not an avaluation, they mean "Dont know how to answer" and "Does not apply" respectively. So these values were substituted to null values.

In [ ]:
questions = [
    "qe_i27",
    "qe_i28",
    "qe_i29",
    "qe_i30",
    "qe_i31",
    "qe_i32",
    "qe_i33",
    "qe_i34",
    "qe_i35",
    "qe_i36",
    "qe_i37",
    "qe_i38",
    "qe_i39",
    "qe_i40",
    "qe_i41",
    "qe_i42",
    "qe_i43",
    "qe_i44",
    "qe_i45",
    "qe_i46",
    "qe_i47",
    "qe_i48",
    "qe_i49",
    "qe_i50",
    "qe_i51",
    "qe_i52",
    "qe_i53",
    "qe_i54",
    "qe_i55",
    "qe_i56",
    "qe_i57",
    "qe_i58",
    "qe_i59",
    "qe_i60",
    "qe_i61",
    "qe_i62",
    "qe_i63",
    "qe_i64",
    "qe_i65",
    "qe_i66",
    "qe_i67",
    "qe_i68",
]
for column in questions:
    sq_df = sq_df.withColumn(
        column,
        when((sq_df[column] == 7) | (sq_df[column] == 8), None).otherwise(
            sq_df[column]
        ),
    )

Then, the answers were grouped so each row has the mean of all the aswers to each course

In [ ]:
grouped_sq_df = sq_df.groupBy(["nu_ano", "co_curso"]).agg(
    *[avg(col).alias(f"{col}").cast(DecimalType(10, 3)) for col in questions]
)

# Join Dataframes

Since the coursed dataframe and this dataframe depends on the same primary key (year and course_code), they were joined.

In [ ]:
grouped_sq_df = grouped_sq_df.withColumnRenamed("nu_ano", "ano").withColumnRenamed(
    "co_curso", "codigo_do_curso"
)

In [ ]:
curso_df = curso_df.join(grouped_sq_df, on=["ano", "codigo_do_curso"], how="inner")

In [ ]:
curso_df.printSchema()

root
 |-- ano: integer (nullable = true)
 |-- codigo_do_curso: integer (nullable = true)
 |-- codigo_da_area: integer (nullable = true)
 |-- grau_academico: string (nullable = true)
 |-- codigo_da_ies: integer (nullable = true)
 |-- codigo_do_municipio: integer (nullable = true)
 |-- modalidade_de_ensino: string (nullable = true)
 |-- no_de_concluintes_inscritos: integer (nullable = true)
 |-- no_de_concluintes_participantes: integer (nullable = true)
 |-- nota_bruta_fg: decimal(10,3) (nullable = true)
 |-- nota_padronizada_fg: decimal(10,3) (nullable = true)
 |-- nota_bruta_ce: decimal(10,3) (nullable = true)
 |-- nota_padronizada_ce: decimal(10,3) (nullable = true)
 |-- conceito_enade_continuo: decimal(10,3) (nullable = true)
 |-- conceito_enade_faixa: integer (nullable = true)
 |-- qe_i27: decimal(10,3) (nullable = true)
 |-- qe_i28: decimal(10,3) (nullable = true)
 |-- qe_i29: decimal(10,3) (nullable = true)
 |-- qe_i30: decimal(10,3) (nullable = true)
 |-- qe_i31: decimal(10,3) (n

In [ ]:
result = curso_df.collect()[0].asDict()

for key, value in result.items():
    print(f"{key}: {value}")

ano: 2022
codigo_do_curso: 58120
codigo_da_area: 2
grau_academico: Bacharelado
codigo_da_ies: 1591
codigo_do_municipio: 1600303
modalidade_de_ensino: Educação Presencial
no_de_concluintes_inscritos: 156
no_de_concluintes_participantes: 99
nota_bruta_fg: 50.143
nota_padronizada_fg: 1.679
nota_bruta_ce: 33.776
nota_padronizada_ce: 1.490
conceito_enade_continuo: 1.537
conceito_enade_faixa: 2
qe_i27: 4.959
qe_i28: 4.752
qe_i29: 4.562
qe_i30: 4.220
qe_i31: 5.008
qe_i32: 4.395
qe_i33: 5.048
qe_i34: 5.073
qe_i35: 5.024
qe_i36: 4.992
qe_i37: 4.339
qe_i38: 4.379
qe_i39: 4.581
qe_i40: 4.008
qe_i41: 4.815
qe_i42: 4.806
qe_i43: 3.816
qe_i44: 3.509
qe_i45: 4.286
qe_i46: 3.423
qe_i47: 4.361
qe_i48: 4.161
qe_i49: 4.508
qe_i50: 4.270
qe_i51: 4.395
qe_i52: 2.733
qe_i53: 2.441
qe_i54: 4.615
qe_i55: 4.746
qe_i56: 3.992
qe_i57: 4.664
qe_i58: 4.528
qe_i59: 4.317
qe_i60: 3.819
qe_i61: 4.721
qe_i62: 4.615
qe_i63: 4.583
qe_i64: 4.446
qe_i65: 4.934
qe_i66: 4.667
qe_i67: 3.843
qe_i68: 4.901


# Save Dataframes

All tables were saved for analysis

In [ ]:
curso_df.write.format("delta").mode("overwrite").saveAsTable("curso")
ies_df.write.format("delta").mode("overwrite").saveAsTable("ies")
municipio_df.write.format("delta").mode("overwrite").saveAsTable("municipio")
area_df.write.format("delta").mode("overwrite").saveAsTable("area")